<center><h1>Предобработка текста и определение языка</h1></center>


### Вариант А

Написать классификатор для автоматического определения языка текста.
Данные:
Выбрать 4 языка с похожими алфавитами.
Набрать обуч. выборку — (хотя бы) по 100 статей из википедии на каждый язык. Бонус: скачать дампы википедии для выбранных языков, почистить и использовать в качестве обучающей выборки их.
Проверочная выборка: скачать (хотя бы) по 30 статей из википедии на каждый язык.

Признаки для обучения (векторизация документов): символьные триграммы.

Предобработка:
Удалить из всех текстов символы/токены/части документов, которые добавят шума на этапе векторизации или затруднят классификацию.
Выбрать по 3 статьи для каждого языка и подробно описать свои решения по препроцессингу.

Векторизовав тексты, построить матрицу схожести текстов, визуализировать её.

Какие языки оказались очень похожи друг на друга, а какие нет? Иными словами, какие языки проще различить по нграммам, а какие труднее? Выбрать один язык. С какими языками его легко спутать (по вашей векторизации)?

Обучить классификатор (например, NaiveBayes или SVM).
Оценить качество работы системы: посчитать точность, полноту, f-меру.

Сколько обучающих данных достаточно для получения хороших результатов? Бонус: график зависимости accuracy от объёма обучающей выборки.
Сравните качество класификации на униграммах, биграммах, 3-граммах, 4-граммах, 5-граммах. Бонус: постройте такой график. 
Какие именно нграммы оказались наиболее полезными признаками? (например, содержащие символы, которые используются лишь в одно алфавите?)
Интерпретировать результаты.

Оформление: в свой репозиторий загрузить папку langdetect_a_hw, в которой должны быть:
lang_detect_results.txt (или .md или .pdf) с описанием всех ваших шагов (сколько каких текстов взяли, какие методы использовали, какие результаты получились). Описание шага предобработки — обязательно;
проверочная выборка, на которой вы тестировали классификатор (или, что предпочтительнее, файл с ссылками на статьи википедии, которые вы использовали);
код в .ipynb либо .py;


### Вариант B

Данные: всеобщая декларация прав человека (см. то, что делали на занятии).
Бонус: выкачать по 100 статей из википедии для 7 языков и использовать их вместо маленького датасета.

Предобработка: 
Удалить из всех текстов символы/токены/части документов, которые затруднят классификацию.
Выбрать по 3 текста для каждого языка и подробно описать свои решения по препроцессингу.

Собрать для каждого языка частотный список слов/биграмм/триграмм. Записать топ-300 в текстовый файл (по одному нграмму на строке, от более частого к менее частому).
Построить матрицу схожести языков, визуализировать её. Схожесть считаем как объём пересечения полученных топов нграмм (сколько нграммов попали в оба топа?).
Какие языки оказались очень похожи друг на друга, а какие нет? Иными словами, какие языки проще различить по нграммам, а какие труднее? Интерпретируйте результат.
Оформление: в свой репозиторий загрузить папку langdetect_b_hw, в которой должны быть:
описание шага предобработки — обязательно;
7 файлов с наиболее частотными нграммами (например, ru.txt, uk.txt)
визуализация матрицы схожести (например, similarity_matrix.png)
интерпретация результатов (lang_detect_results.txt или .md или .pdf)
код в .ipynb либо .py

Бонус: в википедии есть статья о Декларации (на разных языках). Скопируйте тексты этих статей в txt файлы (например, decl_ru.txt) и определите автоматически язык статьи: посчитайте топ слов/нграмм, сравните объём пересечения этого топа с топами для разных языков, считайте за ответ наиболее похожий. Оцените качество такой системы. Как изменится качество распознавания языка, если проверять не на всей статье, а лишь на одном предложении/абзаце?







In [2]:
import wikipedia
from collections import defaultdict
from time import sleep
from pickle import dump, load
from itertools import chain
from pandas import DataFrame
import re

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [72]:
languages = ['en', 'fr', 'cz', 'es']
pages, labels = defaultdict(lambda: []), defaultdict(lambda: [])

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def normalize(text):
    reg_exps = (re.compile('[^\w ]+'), re.compile('[\d]+'), re.compile('[ ]+'))
    for reg in reg_exps:
        text = reg.sub(' ', text)
    return text.strip()

In [73]:
for language_id, language in enumerate(languages):
    wikipedia.set_lang(language)
    for page_title in wikipedia.random(10):
        title = page_title
        if 'disambig' in page_title: 
            title = page_title.split(' ')[0] 
        while(True):
            try:
                normalized_text = normalize(wikipedia.page(str(wikipedia.search(title)[0])).content)
                pages[language].append(normalized_text)
                labels[language].append(language_id)
                break
            except wikipedia.DisambiguationError:
                title = wikipedia.random(1)

In [82]:
data = DataFrame(list(zip(list(chain.from_iterable(pages[language] for language in languages)), 
                          list(chain.from_iterable(labels[language] for language in languages)))),
                          columns=['text', 'language'])

In [85]:
data.to_csv('wikipedia_languages.csv')

In [7]:
data = DataFrame.from_csv('wikipedia_languages.csv', encoding='utf-8')

In [8]:
train, test = train_test_split(data, test_size=0.3)

In [11]:
select = SelectPercentile(percentile=100)
clf = LogisticRegression(tol=1e-8, penalty='l2', C=1)
svm1 = svm.LinearSVC()
svm2 = svm.SVC()
countvect_char_wb =[CountVectorizer(ngram_range=(3, 3), analyzer='char_wb'), TfidfVectorizer(ngram_range=(1, 3), analyzer='char', binary=False)]

In [12]:
for countvect in countvect_char_wb:
    char_model = Pipeline([('vect', countvect), ('select', select), ('logr', clf)])
    char_model.fit(train.text.values, train.language.values)
    print(f1_score(char_model.predict(test.text.values), test.language.values, average='micro'))

1.0
0.583333333333
